In [32]:
import json
import subprocess
import os
from io import StringIO
import pandas as pd

In [3]:
json_path = "/Users/eliasberger/Documents/Uni/4_Semester/Hyperautomation/Generating-Terraform-configuration-files/data/train-clean/train/json-fixer.txt"

json_file = json.load(open(json_path))

In [4]:
json_file[0]

{'repo_name': 'mrllama123/terraform-scripts',
 'path': 'digital-ocean-multiSalt/outputs.tf',
 'copies': 2,
 'size': 169,
 'content': '\noutput "Public ip main" {\n  value = "${digitalocean_droplet.mainServer.ipv4_address}"\n}\n\noutput "Name main " {\n  value = "${digitalocean_droplet.mainServer.name}"\n}\n\n\n\n',
 'hash': 660047427209366107,
 'line_mean': 14.3636363636,
 'line_max': 59,
 'alpha_frac': 0.6804733728,
 'autogenerated': False}

In [16]:
print(json_file[10]["content"])

variable "pvt_key" {
	description = "Path to your private key"
	default = "~/.ssh/id_rsa"
}

variable "do_token" {
	description = "Your Digital Ocean API token"
	default = ""
}

variable "do_ssh_fingerprint" {
	description = "The fingerprint of your Digital Ocean SSH Key"
	default = ""
}

variable "server_name" {
	description = "What do you want to tag / name your servers as?"
    default = "server"
}

variable "azure_settings_file" {
    default = "settings.publishsettings"
}

variable "azure_ssh_key_thumbprint" {
	description = "The fingerprint of your Azure SSH Key"
	default = ""
}

variable "hosted_service" {
	description = "Which hosted service would you like to launch your VM in?"
	default = ""
}

variable "storage_service" {
	description = "Which storeage would you like to store your VM in?"
	default = ""
}

variable "azure_username" {
	description = "What username would you like to create on your VM?"
	default = "azureuser"
}

variable "aws_access_key" {
	description = "Your AW

In [54]:
temp_tf_file = "/Users/eliasberger/Documents/Uni/4_Semester/Hyperautomation/Generating-Terraform-configuration-files/data/train-clean/train/temp.tf"
temp_tf_dir = "/Users/eliasberger/Documents/Uni/4_Semester/Hyperautomation/Generating-Terraform-configuration-files/data/train-clean/train/"
csv = StringIO()
files_affected = 0
for i in range(2400):
    f = open(temp_tf_file, "w")
    # write content to file
    if i > len(json_file):
        break
    f.write(json_file[i]["content"])
    f.close()
    result = subprocess.run(["tfsec", f"{str(temp_tf_dir)}", "-f", "csv"], capture_output=True)
    csv.write(result.stdout.decode("utf-8"))
    if len(result.stdout.decode("utf-8")) > 66:
        files_affected += 1


In [59]:
files_affected/2400

0.29

In [58]:
csv.seek(0)
df = pd.read_csv(csv)
df_filter = df[~df["file"].str.contains("file")]
df_filter.to_csv(f"data/security_smells_github.csv")

In [44]:
df_filter

,file,start_line,end_line,rule_id,severity,description,link,passed
3,temp.tf,38,49,google-compute-enable-vpc-flow-logs,LOW,Subnetwork does not have VPC flow logs enabled.,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false
4,temp.tf,24,35,google-compute-enable-vpc-flow-logs,LOW,Subnetwork does not have VPC flow logs enabled.,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false
9,temp.tf,18,21,aws-ec2-enable-at-rest-encryption,HIGH,Root block device is not encrypted.,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false
10,temp.tf,13,16,aws-ec2-enable-at-rest-encryption,HIGH,Root block device is not encrypted.,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false
11,temp.tf,18,21,aws-ec2-enforce-http-token-imds,HIGH,Instance does not require IMDS access to requi...,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false
...,...,...,...,...,...,...,...,...
468,temp.tf,1,7,aws-efs-enable-at-rest-encryption,HIGH,File system is not encrypted.,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false
471,temp.tf,37,44,aws-ec2-add-description-to-security-group-rule,LOW,Security group rule does not have a description.,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false
472,temp.tf,47,62,aws-ec2-enable-launch-config-at-rest-encryption,HIGH,Root block device is not encrypted.,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false
473,temp.tf,47,62,aws-ec2-enforce-launch-config-http-token-imds,HIGH,Launch configuration does not require IMDS acc...,https://aquasecurity.github.io/tfsec/v1.28.1/c...,false


In [38]:
csv.seek(0)
csv.getvalue()

'file,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\nfile,start_line,end_line,rule_id,severity,description,link,passed\n'